In [1]:
""" Store area of 30sPfaf06 basins in postGIS database. 
-------------------------------------------------------------------------------

Store the area of 30sPfaf06 zones in the postGIS table. 


Args:
    TESTING (Boolean) : Toggle testing case.
    SCRIPT_NAME (string) : Script name.
    OUTPUT_VERSION (integer) : output version.

"""

TESTING = 1
OVERWRITE = 1 
SCRIPT_NAME = "Y2018M05D22_RH_Store_Area_PostGIS_30sPfaf06_V01"
OUTPUT_VERSION = 1

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"
#TABLE_NAME = "area30spfaf06v01"

S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2018M04D20_RH_Zonal_Stats_Area_EE_V01/output_V02"

ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)

print("\nInput ec2: " + ec2_input_path,
      "\nInput s3: " + S3_INPUT_PATH)




Input ec2: /volumes/data/Y2018M05D22_RH_Store_Area_PostGIS_30sPfaf06_V01/input_V01 
Input s3: s3://wri-projects/Aqueduct30/processData/Y2018M04D20_RH_Zonal_Stats_Area_EE_V01/output_V02


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M05D22 UTC 12:18


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

In [4]:
!rm -r {ec2_input_path}
!mkdir -p {ec2_input_path}

In [5]:
!aws s3 cp {S3_INPUT_PATH} {ec2_input_path} --recursive --exclude="*" --include="*.pkl"

download: s3://wri-projects/Aqueduct30/processData/Y2018M04D20_RH_Zonal_Stats_Area_EE_V01/output_V02/df_hybas_lev06_5min.pkl to ../../../../data/Y2018M05D22_RH_Store_Area_PostGIS_30sPfaf06_V01/input_V01/df_hybas_lev06_5min.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2018M04D20_RH_Zonal_Stats_Area_EE_V01/output_V02/df_hybas_lev06_30s.pkl to ../../../../data/Y2018M05D22_RH_Store_Area_PostGIS_30sPfaf06_V01/input_V01/df_hybas_lev06_30s.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2018M04D20_RH_Zonal_Stats_Area_EE_V01/output_V02/df_hybas_lev00_30s.pkl to ../../../../data/Y2018M05D22_RH_Store_Area_PostGIS_30sPfaf06_V01/input_V01/df_hybas_lev00_30s.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2018M04D20_RH_Zonal_Stats_Area_EE_V01/output_V02/df_hybas_lev00_5min.pkl to ../../../../data/Y2018M05D22_RH_Store_Area_PostGIS_30sPfaf06_V01/input_V01/df_hybas_lev00_5min.pkl


In [6]:
import os
import numpy as np
import pandas as pd
import aqueduct3
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)

In [7]:
engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))
connection = engine.connect()

"""
if OVERWRITE:
    sql = text("DROP TABLE IF EXISTS {};".format(TABLE_NAME))
    result = engine.execute(sql)
"""

'\nif OVERWRITE:\n    sql = text("DROP TABLE IF EXISTS {};".format(TABLE_NAME))\n    result = engine.execute(sql)\n'

In [8]:
input_file_names = os.listdir(ec2_input_path)

In [9]:
input_file_names

['df_hybas_lev00_5min.pkl',
 'df_hybas_lev06_5min.pkl',
 'df_hybas_lev06_30s.pkl',
 'df_hybas_lev00_30s.pkl']

In [10]:
def pre_process_df(df,hybas_level,spatial_resolution):
    df["zones"] = df["zones"].astype(np.int64)
    df["area_m2_{}pfaf{:02.0f}".format(spatial_resolution,hybas_level)] = df["sum"]
    df.drop("sum",axis=1,inplace=True)
    df.sort_index(axis=1, inplace=True)
    df.set_index("zones",inplace=True)
    df.index.name = "pfafid_{}pfaf{:02.0f}".format(spatial_resolution,hybas_level)
    return df

In [11]:
hybas_levels = [0,6]
spatial_resolutions = ["30s","5min"]

In [12]:
for hybas_level in hybas_levels:
    for spatial_resolution in spatial_resolutions:
        input_file_name = "df_hybas_lev{:02.0f}_{}.pkl".format(hybas_level,spatial_resolution)
        input_file_path = "{}/{}".format(ec2_input_path,input_file_name)
        df = pd.read_pickle(input_file_path)
        df_cleaned = pre_process_df(df,hybas_level,spatial_resolution)
        table_name = "area_m2_{}pfaf{:02.0f}".format(spatial_resolution,hybas_level)
        df_cleaned.to_sql(table_name,engine,if_exists='replace', index=True,chunksize=100)

In [13]:
df_cleaned.head()

,area_m2_5minpfaf06,count,output_version,parameter,pfaf_level,reducer,script_used,spatial_aggregation,spatial_resolution,unit
pfafid_5minpfaf06,,,,,,,,,,
111011,1.933489e+09,26,2,area,6,sum,Y2018M04D20_RH_Zonal_Stats_Area_EE_V01,hydrobasin,5min,m2
111012,2.910369e+09,39,2,area,6,sum,Y2018M04D20_RH_Zonal_Stats_Area_EE_V01,hydrobasin,5min,m2
111013,8.220865e+08,11,2,area,6,sum,Y2018M04D20_RH_Zonal_Stats_Area_EE_V01,hydrobasin,5min,m2
111014,4.127270e+09,55,2,area,6,sum,Y2018M04D20_RH_Zonal_Stats_Area_EE_V01,hydrobasin,5min,m2
111015,1.661628e+10,219,2,area,6,sum,Y2018M04D20_RH_Zonal_Stats_Area_EE_V01,hydrobasin,5min,m2


In [14]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:14:43.240018


Previous Runs:  
0:14:43.240018
